In [79]:
import os

def list_files(start_directory):
    for dirpath, dirs, files in os.walk(start_directory):
        for dir_ in dirs:
            print("dir:", os.path.join(dirpath, dir_))
        for file in files:
            print(os.path.join(dirpath, file))

def get_files(start_directory):
    for dirpath, dirs, files in os.walk(start_directory, topdown=True):
        for file in files:
            if file.endswith(".py"):
                yield os.path.join(dirpath, file)



In [14]:
search_directory = "/Users/deepan/workspace/temp/"

files = [(file, os.path.getctime(file)) for file in get_files(search_directory)]
files_sorted_by_creation_time = sorted(files, key=lambda x: x[1])
for file, ctime in files_sorted_by_creation_time:
    print(file, ctime)

/Users/deepan/workspace/temp/bst/bst.py 1520101243.6053371
/Users/deepan/workspace/temp/icake/q1.py 1520101261.6830838
/Users/deepan/workspace/temp/icake/q2.py 1520101261.6862657
/Users/deepan/workspace/temp/icake/test_q1.py 1520101261.6887019
/Users/deepan/workspace/temp/icake/test_q2.py 1520101261.6909235
/Users/deepan/workspace/temp/icake/__init__.py 1520101359.1391308
/Users/deepan/workspace/temp/.DS_Store 1520101362.8405874
/Users/deepan/workspace/temp/bst/__init__.py 1520101362.8839607
/Users/deepan/workspace/temp/icake/__init__.pyc 1520101370.5520444
/Users/deepan/workspace/temp/icake/test_q1.pyc 1520101370.5527544
/Users/deepan/workspace/temp/icake/q1.pyc 1520101370.5532155
/Users/deepan/workspace/temp/icake/test_q2.pyc 1520101370.5547574
/Users/deepan/workspace/temp/icake/q2.pyc 1520101370.5556128
/Users/deepan/workspace/temp/bst/__init__.pyc 1520101370.5564766
/Users/deepan/workspace/temp/icake/__pycache__/__init__.cpython-36.pyc 1520101955.2695024
/Users/deepan/workspace/tem

In [91]:
import hashlib

def find_duped_files(start_directory):
    files_seen = {}
    duplicates = []
    for cur_file in get_files(start_directory):
        file_hash = _compute_hash(cur_file)
        if file_hash in files_seen:
            # find the duplicate
            prev_file = files_seen[file_hash]
            prev_file_creation_time = os.path.getctime(prev_file)
            cur_file_creation_time = os.path.getctime(cur_file)
            if prev_file_creation_time > cur_file_creation_time:
                # file1 is latest
                duplicates.append((prev_file, cur_file))
            else:
                duplicates.append((cur_file, prev_file))
        else:
            # Hitting first time. Add it to the seen map
            files_seen[file_hash] = cur_file
    return duplicates

def _compute_hash(file):
    sha1_hasher = hashlib.sha256()
    with open(file, "r+") as f:
        for line in f:
            sha1_hasher.update(line.encode())
    return sha1_hasher.hexdigest()
        

In [81]:
print(find_duped_files(search_directory))
# print(search_directory.encode())

Hash for file: /Users/deepan/workspace/temp/file2.py
Hash for file: /Users/deepan/workspace/temp/dup_file1.py
Hash for file: /Users/deepan/workspace/temp/file3.py
Hash for file: /Users/deepan/workspace/temp/file1.py
Hash for file: /Users/deepan/workspace/temp/dup_file2.py
Hash for file: /Users/deepan/workspace/temp/diskobjs.py
Hash for file: /Users/deepan/workspace/temp/icake/q1_dup.py
Hash for file: /Users/deepan/workspace/temp/icake/q1.py
Hash for file: /Users/deepan/workspace/temp/icake/test_q1.py
Hash for file: /Users/deepan/workspace/temp/icake/__init__.py
Hash for file: /Users/deepan/workspace/temp/icake/test_q2.py
Hash for file: /Users/deepan/workspace/temp/icake/q2.py
Hash for file: /Users/deepan/workspace/temp/bst/test_bst.py
Hash for file: /Users/deepan/workspace/temp/bst/bst.py
Hash for file: /Users/deepan/workspace/temp/bst/__init__.py
[('/Users/deepan/workspace/temp/dup_file1.py', '/Users/deepan/workspace/temp/file1.py'), ('/Users/deepan/workspace/temp/dup_file2.py', '/Use

In [96]:
import time
working_dir = os.getcwd() # "/Users/deepan/workspace/temp/"
# Create three files
test_files = ["file1.py", "file2.py", "file3.py"]
for test_file in test_files:
    with open(os.path.join(working_dir, test_file), "w+") as f:
        f.write("print({} says Hello, world)".format(test_file))
        # Artifical sleep to differentiate creation time
        time.sleep(1)
        f.close()

# Create duplicate files for the first two files
dup_files = ["dup_" + file for file in test_files[:2]]
for test_file in dup_files:
     with open(os.path.join(working_dir, test_file), "w+") as f:
        f.write("print({} says Hello, world)".format(test_file))
        time.sleep(1)
        f.close()

# Verify duplicates
exp_duped_files = set()
exp_duped_files.add((os.path.join(working_dir,"dup_file1.py"), os.path.join(working_dir,"file1.py")))
exp_duped_files.add((os.path.join(working_dir,"dup_file2.py"), os.path.join(working_dir,"file2.py")))
act_dup_files = find_duped_files(working_dir)
for dup_file in act_dup_files:
    assert dup_file in exp_duped_files
    

for test_file in test_files + dup_files:
    os.remove(os.path.join(working_dir, test_file))

    

In [117]:
from collections import deque

def merge_sorted_lists_using_deque(left, right):
    # Edge cases
    # Both lists empty
    # One of the lists is empty

    # neither list is empty
    # begin merging
    if not left and not right:
        return []
    
    if left and not right:
        print("Empty right")
        return left
    
    if not left and right:
        print("Empty left")
        return right
    
    merged = deque()

    while (len(left) > 0 and len(right) > 0):
        left_item = left.pop()
        right_item = right.pop()

        min_item = min(left_item, right_item)
        max_item = max(left_item, right_item)

        maxlist = left if max_item == left_item else right
        # minlist = left if maxlist == right else right

        # Enqueue the max into the merged list
        merged.appendleft(max_item)

        while (len(maxlist) > 0 and maxlist[-1] > min_item):
            merged.appendleft(maxlist.pop())
        
        merged.appendleft(min_item)
    
    # Add remaining items from non-empty list
    assert (len(left) == 0 or len(right) == 0)
    for i in range(len(left)-1, -1, -1):
        merged.appendleft(left[i])
    for i in range(len(right)-1, -1, -1):
        merged.appendleft(right[i])
    
    return merged

In [134]:
def merge_sorted_lists(left, right):
    """ Merge two sorted lists using a deque """
    
    # Edge cases
    if not left and not right:
        print("Both left and right are empty")
        return []
    
    if left and not right:
        return left
    
    if not left and right:
        return right

    merged = []

    left_index = 0
    right_index = 0
    left_max = len(left)
    right_max = len(right)
 
    while (left_index < left_max and right_index < right_max): # end of list is not reached
        # left_item = left[left_index] # Get the first item from left
        # right_item = right[right_index] # Get the first item from right

        # min_item = min(left_item, right_item)
        max_item = max(left[left_index], right[right_index])
      
        # Add items from the min list
        if left[left_index] < right[right_index]:
            while (left_index < left_max and left[left_index] < right[right_index]):
                merged.append(left[left_index])
                left_index += 1
            right_index += 1
        else:
            while (right_index < right_max and right[right_index] < left[left_index]):
                merged.append(right[right_index])
                right_index += 1
            left_index += 1
        
        merged.append(max_item)

    
    # Add remaining items from non-empty list
    assert (left_max == left_index or right_max == right_index)
    for i in range(left_index, left_max):
        merged.append(left[i])
    for i in range(right_index, right_max):
        merged.append(right[i])
    
    return merged

In [130]:
left = [1, 3, 5, 7]
right = [2, 4, 6, 9]

print(merge_sorted_lists_using_deque(left, right))

deque([1, 2, 3, 4, 5, 6, 7, 9])
Both left and right are empty
[]


In [144]:
import heapq

left =  [-3,  4,  6, 10, 11, 15]
right = [-1,  5,  8, 12, 14, 19]

%timeit print(merge_sorted_lists(left, right))
%timeit print(list(heapq.merge(left, right)))

[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4

[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4

[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4

[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4

[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4

...

[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4

[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4

[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4

[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4

[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
[-3, -1, 4